<a href="https://colab.research.google.com/github/TarekHasan011/Pattern-Recognition/blob/main/Na%C3%AFve_Bayes_Pattern_Recognition_Lab_Offline_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Reading csv file
import pandas as pd
data = pd.read_csv('tic-tac-toe.csv')

In [ ]:
label_column_name = 'Class'
X = data.loc[:, data.columns != label_column_name]
y = data[label_column_name]

In [ ]:
#Preprocessing
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for column in X.columns:
    X[column] = le.fit_transform(X[column])

class_label = {
    'negative' : 0,
    'positive' : 1
}
y = y.map(class_label)

In [ ]:
#Split Data (train and test)
from sklearn.model_selection import train_test_split
#without stratification
train_X, test_X, train_y, test_y = train_test_split(X,y,test_size=0.1)
#with stratification
s_train_X, s_test_X, s_train_y, s_test_y = train_test_split(X,y,test_size=0.1,stratify=y)

In [ ]:
import numpy as np
learning_result = {}

def fit(features,labels):
    types_of_value = {}
    for column in features.columns:
        types_of_value[column] = len(set(features[column]))
    types_of_value[label_column_name] = len(set(labels))

    learning_result.clear()
    total_data = len(labels)
    # for prior probabilities : class -> probability
    # for posterior probabilities : (feature, feature value, class level) -> probability  
    for i in range(max(list(labels))+1):
        learning_result[i] = (list(labels).count(i)+1) / (total_data+types_of_value[label_column_name])
    

    labels = np.array(labels)

    for x in features.columns:
        temp = set()
        i = 0
        for f in features[x]:
            c = labels[i]
            if learning_result.get((x,f,c)) == None:
                learning_result[(x,f,c)] = 1
            else:
                learning_result[(x,f,c)] += 1

            temp.add((x,f,c))
            i+=1

        for item in temp:
            learning_result[item] = (learning_result[item]+1) / (list(labels).count(item[2])+types_of_value[item[0]])

    return



In [ ]:
def predict(features):
    features_name = features.columns
    features_name = np.array(features_name)
    
    predicted_result = []
    features = np.array(features)

    for x in features:
        maximum_result = 0
        level = 0
        for c in class_label:
            prod = learning_result[class_label[c]]
            for i in range(len(x)):
                prod *= learning_result[(features_name[i],x[i],class_label[c])]
            if prod > maximum_result:
                maximum_result = prod
                level = class_label[c]
        predicted_result.append(level)
    return predicted_result


In [ ]:
fit(train_X,train_y)
predict_y = predict(test_X)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
#without stratification
accuracy = accuracy_score(list(test_y),predict_y)
precision = precision_score(list(test_y),predict_y)
recall = recall_score(list(test_y),predict_y)
f1 = f1_score(list(test_y),predict_y)
print(f'{accuracy} {precision} {recall} {f1}')

0.6979166666666666 0.7361111111111112 0.8412698412698413 0.7851851851851852


In [ ]:
fit(s_train_X,s_train_y)
s_predict_y = predict(s_test_X)

In [ ]:
#with stratification
s_accuracy = accuracy_score(list(s_test_y), s_predict_y)
s_precision = precision_score(list(s_test_y), s_predict_y)
s_recall = recall_score(list(s_test_y), s_predict_y)
s_f1 = f1_score(list(s_test_y), s_predict_y)
print(f'{s_accuracy} {s_precision} {s_recall} {s_f1}')

0.75 0.76 0.9047619047619048 0.8260869565217391


In [ ]:
#using sklearn for comparing
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(s_train_X, s_train_y)
result = gnb.predict(s_test_X)
print(f'{accuracy_score(result,s_test_y)} {precision_score(result, s_test_y)} {recall_score(result, s_test_y)} {f1_score(result, s_test_y)}')

0.7291666666666666 1.0 0.7078651685393258 0.8289473684210525


In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(s_train_X, s_train_y)
clf_result = clf.predict(s_test_X)
print(f'{accuracy_score(clf_result,s_test_y)} {precision_score(clf_result, s_test_y)} {recall_score(clf_result, s_test_y)} {f1_score(clf_result, s_test_y)}')

0.65625 1.0 0.65625 0.7924528301886793
